# Агрегирование и группировка

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

In [ ]:
planets.head()

In [ ]:
rng = np.random.RandomState(1)
ser = pd.Series(rng.rand(5))
ser

In [ ]:
ser.sum()

In [ ]:
ser.mean()

In [ ]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

In [ ]:
df.mean()

In [ ]:
df.mean(axis='columns')

In [ ]:
planets.dropna().describe()

Встроенные функции агрегирования

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

Применимы и к ``DataFrame`` и к ``Series``.

## GroupBy: Разбиение, применение, комбинация (split, apply, combine)

![](split-apply-combine.png)

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

In [ ]:
df.groupby('key')

In [ ]:
df1 = pd.DataFrame({'key1': ['A', 'B', 'C', 'A', 'B', 'C'],'key2': ['A1', 'B1', 'C1', 'A1', 'B1', 'C1'],
                    'key3': ['A3', 'B3', 'C3', 'A3', 'B3', 'C3'],
                   'data': range(6),'data2': range(1,7)}, columns=['key1', 'key2', 'key3', 'data','data2'])

df1.groupby(['key1','key2']).sum()

In [ ]:
df1.groupby('key1').aggregate({'data': 'sum', 'data2': 'max'})

Объект ``DataFrameGroupBy`` проводит вычисления "лениво":

In [ ]:
df.groupby('key').sum()

In [ ]:
planets.groupby('method')

Этот объект можно рассматривать как словарь с ключем по колонкам

In [ ]:
planets.groupby('method')['orbital_period']

In [ ]:
planets.groupby('method')['orbital_period'].median()

#### Итерирование по группам

In [ ]:
for (method, group) in planets.groupby('method'):
    print(f"{method:30} shape={group.shape}")

In [ ]:
planets.groupby('method')['year'].describe()

### Методы aggregate, filter, transform, apply

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

display(df, df.groupby('key').std(), df.groupby('key').filter(filter_func))

In [ ]:
df.groupby('key').transform(lambda x: x - x.mean())

In [ ]:
def norm_by_data2(x):
    # x - это DataFrame сгруппированных значений!
    x['data1'] /= x['data2'].sum()
    return x

display(df, df.groupby('key').apply(norm_by_data2))

### Определение ключа для разбиения

In [ ]:
L = [0, 1, 0, 1, 2, 0]
display(df, df.groupby(L).sum())

In [ ]:
display(df, df.groupby(df['key']).sum())

In [ ]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display(df2, df2.groupby(mapping).sum())

#### В разбиении можно использовать любую функцию Python

In [ ]:
display(df2, df2.groupby(str.lower).mean())

In [ ]:
df2.groupby([str.lower, mapping]).mean()

### Пример группировки

In [ ]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)